In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")
version = "3.0"
base_url = f"https://api.e-stat.go.jp/rest/{version}/"

In [2]:
from urllib.parse import urljoin
import requests
import pandas as pd

In [3]:
statslist_endpoint = "app/json/getStatsList"
statslist_url = urljoin(base_url, statslist_endpoint)
statslist_params = {"appId": appId, "surveyYears": 2020, "limit": 100}
statslist_res = requests.get(statslist_url, params=statslist_params)
statslist_out = statslist_res.json()

In [4]:
statslist_out.keys()

dict_keys(['GET_STATS_LIST'])

In [5]:
statslist_out['GET_STATS_LIST'].keys()

dict_keys(['RESULT', 'PARAMETER', 'DATALIST_INF'])

In [6]:
statslist_out['GET_STATS_LIST']['DATALIST_INF'].keys()

dict_keys(['NUMBER', 'RESULT_INF', 'TABLE_INF'])

In [7]:
table_inf = pd.json_normalize(
    statslist_out,
    record_path=["GET_STATS_LIST", "DATALIST_INF", "TABLE_INF"],
    sep="_"
)
table_inf.columns

Index(['@id', 'STATISTICS_NAME', 'CYCLE', 'SURVEY_DATE', 'OPEN_DATE',
       'SMALL_AREA', 'COLLECT_AREA', 'OVERALL_TOTAL_NUMBER', 'UPDATED_DATE',
       'DESCRIPTION', 'STAT_NAME_@code', 'STAT_NAME_$', 'GOV_ORG_@code',
       'GOV_ORG_$', 'TITLE_@no', 'TITLE_$', 'MAIN_CATEGORY_@code',
       'MAIN_CATEGORY_$', 'SUB_CATEGORY_@code', 'SUB_CATEGORY_$',
       'STATISTICS_NAME_SPEC_TABULATION_CATEGORY',
       'STATISTICS_NAME_SPEC_TABULATION_SUB_CATEGORY1',
       'STATISTICS_NAME_SPEC_TABULATION_SUB_CATEGORY2',
       'STATISTICS_NAME_SPEC_TABULATION_SUB_CATEGORY3',
       'TITLE_SPEC_TABLE_CATEGORY', 'TITLE_SPEC_TABLE_NAME',
       'TITLE_SPEC_TABLE_EXPLANATION'],
      dtype='str')

In [8]:
table_inf[["@id", "TITLE_SPEC_TABLE_NAME", "TITLE_$", "OVERALL_TOTAL_NUMBER"]].head()

,@id,TITLE_SPEC_TABLE_NAME,TITLE_$,OVERALL_TOTAL_NUMBER
0,0004014855,6(1).一般政府の部門別勘定,5.付表 6(1).一般政府の部門別勘定,8424
1,0004014897,6(2).一般政府の部門別勘定(GFS) 年度,5.付表 6(2).一般政府の部門別勘定(GFS) 年度,32265
2,0004014898,7.一般政府の機能別支出(COFOG) 年度,5.付表 7.一般政府の機能別支出(COFOG) 年度,11520
3,0004014899,8.一般政府の機能別最終消費支出(COFOG) 年度,5.付表 8.一般政府の機能別最終消費支出(COFOG) 年度,12800
4,0004014900,9.一般政府から家計への移転の明細表(社会保障関係),5.付表 9.一般政府から家計への移転の明細表(社会保障関係),2835


In [9]:
statsDataId = "0002070010"
meta_endpoint = "app/json/getMetaInfo"
meta_url = urljoin(base_url, meta_endpoint)
meta_params = {"appId": appId, "statsDataId": statsDataId}
meta_res = requests.get(meta_url, params=meta_params)
meta = meta_res.json()
meta["GET_META_INFO"]["METADATA_INF"]["TABLE_INF"].keys()

dict_keys(['@id', 'STAT_NAME', 'GOV_ORG', 'STATISTICS_NAME', 'TITLE', 'CYCLE', 'SURVEY_DATE', 'OPEN_DATE', 'SMALL_AREA', 'COLLECT_AREA', 'MAIN_CATEGORY', 'SUB_CATEGORY', 'OVERALL_TOTAL_NUMBER', 'UPDATED_DATE', 'STATISTICS_NAME_SPEC', 'DESCRIPTION', 'TITLE_SPEC'])

In [10]:
meta.keys()

dict_keys(['GET_META_INFO'])

In [11]:
meta["GET_META_INFO"].keys()

dict_keys(['RESULT', 'PARAMETER', 'METADATA_INF'])

In [12]:
meta["GET_META_INFO"]["RESULT"].keys()

dict_keys(['STATUS', 'ERROR_MSG', 'DATE'])

In [13]:
meta["GET_META_INFO"]["METADATA_INF"].keys()

dict_keys(['TABLE_INF', 'CLASS_INF'])

In [14]:
meta["GET_META_INFO"]["METADATA_INF"]["CLASS_INF"].keys()

dict_keys(['CLASS_OBJ'])

In [15]:
meta["GET_META_INFO"]["METADATA_INF"]["CLASS_INF"]["CLASS_OBJ"][0].keys()

dict_keys(['@id', '@name', 'CLASS'])

In [16]:
metadata = meta["GET_META_INFO"]["METADATA_INF"]
overall_total_number = metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
overall_total_number

3916136

In [17]:
[[n, c["@id"], c["@name"]] for n, c in enumerate(metadata["CLASS_INF"]["CLASS_OBJ"])]

[[0, 'tab', '表章項目'],
 [1, 'cat01', '用途分類'],
 [2, 'cat02', '世帯区分'],
 [3, 'cat03', '世帯主の年齢階級'],
 [4, 'area', '地域区分'],
 [5, 'time', '時間軸（月次）']]

In [18]:
metadata["CLASS_INF"]["CLASS_OBJ"][0]["CLASS"]

{'@code': '01', '@name': '金額', '@level': ''}

In [19]:
pd.DataFrame(metadata["CLASS_INF"]["CLASS_OBJ"][1]["CLASS"]).iloc[15:25, :]

,@code,@name,@level,@unit,@parentCode
15,017,農林漁家世帯の割合,1,％,NaN
16,018,受取,1,円,NaN
17,019,実収入,2,円,018
18,020,経常収入,3,円,019
19,021,勤め先収入,4,円,020
20,022,世帯主収入,5,円,021
21,023,世帯主収入（男）,7,円,022
22,024,定期収入,6,円,022
23,267,臨時収入・賞与,6,円,022
24,025,臨時収入,7,円,267


In [20]:
pd.DataFrame(metadata["CLASS_INF"]["CLASS_OBJ"][2]["CLASS"])

,@code,@name,@level
0,03,二人以上の世帯（2000年～）,1
1,04,二人以上の世帯のうち勤労者世帯（2000年～）,1
2,01,"二人以上の世帯（農林漁家世帯を除く）（1985年～2007年,2017年）",1
3,02,"二人以上の世帯のうち勤労者世帯（農林漁家世帯を除く）（1985年～2007年,2017年）",1


In [21]:
pd.DataFrame(metadata["CLASS_INF"]["CLASS_OBJ"][3]["CLASS"]).head()

,@code,@name,@level
0,A00,平均,1
1,425,24歳以下,1
2,435,34歳以下,1
3,205,25～29歳,1
4,206,30～34歳,1


In [22]:
pd.DataFrame(metadata["CLASS_INF"]["CLASS_OBJ"][1]["CLASS"]).head()

,@code,@name,@level,@unit,@parentCode
0,001,世帯数分布（抽出率調整）,1,一万分比,NaN
1,002,集計世帯数,1,世帯,NaN
2,003,世帯人員,1,人,NaN
3,004,18歳未満人員,2,人,003
4,005,65歳以上人員,2,人,003


In [23]:
statsDataId = "0002070010"
data_params = {
    "appId": appId, 
    "statsDataId": statsDataId,
    "lvCat01": "4",  # 用途分類を階層4で絞る
    "cdCat02": "04",  # 世帯区分を二人以上の世帯のうち勤労者世帯（2000年～）で絞る
    "cdCat03": "A00",  # 世帯主の年齢階級を平均で絞る
    "cdTimeFrom": "2020000101",  # 2020年1月以降で絞る
    "cdTimeTo": "2022001212",  #  2022年12月以前で絞る
}

In [24]:
from io import StringIO
csv_data_endpoint = "app/getSimpleStatsData"
csv_data_url = urljoin(base_url, csv_data_endpoint)
csv_data_res = requests.get(csv_data_url, params=data_params)
print(csv_data_res.text[:1000])

"RESULT"
"STATUS","0"
"ERROR_MSG","正常に終了しました。"
"DATE","2026-02-01T15:04:26.836+09:00"
"RESULT_INF"
"TOTAL_NUMBER","1188"
"FROM_NUMBER","1"
"TO_NUMBER","1188"
"TABLE_INF","0002070010"
"STAT_NAME","00200561","家計調査"
"GOV_ORG","00200","総務省"
"STATISTICS_NAME","家計調査 家計収支編 二人以上の世帯"
"TITLE","005","用途分類 用途分類（世帯主の年齢階級別）"
"CYCLE","月次"
"SURVEY_DATE","0"
"OPEN_DATE","2026-01-09"
"SMALL_AREA","0"
"COLLECT_AREA","該当なし"
"MAIN_CATEGORY","07","企業・家計・経済"
"SUB_CATEGORY","04","家計"
"OVERALL_TOTAL_NUMBER","3916136"
"UPDATED_DATE","2026-01-09"
"STATISTICS_NAME_SPEC","家計調査","家計収支編","二人以上の世帯","","",""
"TITLE_SPEC","用途分類","用途分類（世帯主の年齢階級別）","","","",""
"NOTE","***","調査又は集計していないもの"
"NOTE","-","該当数字がないもの"
"NOTE","X","数値が秘匿されているもの"
"VALUE"
"tab_code","表章項目","cat01_code","用途分類","cat02_code","世帯区分","cat03_code","世帯主の年齢階級","area_code","地域区分","time_code","時間軸（月次）","unit","value","annotation"
"01","金額","021","勤め先収入","04","二人以上の世帯のうち勤労者世帯（2000年～）","A00","平均","00000","全国","2020000101","2020年1月","円","458814",""
"01","金額","0

In [25]:
pd.read_csv(StringIO(csv_data_res.text), skiprows=28).head()

,tab_code,表章項目,cat01_code,用途分類,cat02_code,世帯区分,cat03_code,世帯主の年齢階級,area_code,地域区分,time_code,時間軸（月次）,unit,value,annotation
0,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000101,2020年1月,円,458814.0,NaN
1,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000202,2020年2月,円,450451.0,NaN
2,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000303,2020年3月,円,467521.0,NaN
3,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000404,2020年4月,円,458774.0,NaN
4,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000505,2020年5月,円,451416.0,NaN


In [26]:
pd.read_csv(StringIO(csv_data_res.text.split('"VALUE"')[1][1:])).head()

,tab_code,表章項目,cat01_code,用途分類,cat02_code,世帯区分,cat03_code,世帯主の年齢階級,area_code,地域区分,time_code,時間軸（月次）,unit,value,annotation
0,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000101,2020年1月,円,458814.0,NaN
1,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000202,2020年2月,円,450451.0,NaN
2,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000303,2020年3月,円,467521.0,NaN
3,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000404,2020年4月,円,458774.0,NaN
4,1,金額,21,勤め先収入,4,二人以上の世帯のうち勤労者世帯（2000年～）,A00,平均,0,全国,2020000505,2020年5月,円,451416.0,NaN


In [27]:
data_endpoint = "app/json/getStatsData"
data_url = urljoin(base_url, data_endpoint)
data_res = requests.get(data_url, params=data_params)
data = data_res.json()

In [28]:
value_df = pd.json_normalize(
    data,
    record_path=["GET_STATS_DATA", "STATISTICAL_DATA", "DATA_INF", "VALUE"]
)
value_df.head()

,@tab,@cat01,@cat02,@cat03,@area,@time,@unit,$
0,01,021,04,A00,00000,2020000101,円,458814
1,01,021,04,A00,00000,2020000202,円,450451
2,01,021,04,A00,00000,2020000303,円,467521
3,01,021,04,A00,00000,2020000404,円,458774
4,01,021,04,A00,00000,2020000505,円,451416


In [29]:
value_df = value_df.rename(
    columns=lambda col: col.lstrip("@").replace("$", "value")
)

In [30]:
value_df.head()

,tab,cat01,cat02,cat03,area,time,unit,value
0,01,021,04,A00,00000,2020000101,円,458814
1,01,021,04,A00,00000,2020000202,円,450451
2,01,021,04,A00,00000,2020000303,円,467521
3,01,021,04,A00,00000,2020000404,円,458774
4,01,021,04,A00,00000,2020000505,円,451416


In [31]:
note = data["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["NOTE"]
note

[{'@char': '***', '$': '調査又は集計していないもの'},
 {'@char': '-', '$': '該当数字がないもの'},
 {'@char': 'X', '$': '数値が秘匿されているもの'}]

In [32]:
import numpy as np

In [33]:
note_char = [n["@char"] for n in note]
value_df = value_df.assign(
    **{
        "value": lambda df: df["value"]
            .replace(note_char, np.nan)
            .astype(float)
    }
)

In [34]:
from typing import List, Dict, Union

In [35]:
def missing_to_nan(value: pd.DataFrame, note: Union[Dict[str, str], List[Dict[str, str]]]) -> pd.DataFrame:
    if isinstance(note, list):
        note_char = [n["@char"] for n in note]
    elif isinstance(note, dict):
        note_char = note["char"]
    else:
        print(f"引数noteの型は辞書またはリスト。noteの型: {type(note)}")
        return value
    return value.assign(
        **{
            "value": lambda df: df["value"]
                .replace(note_char, np.nan)
                .astype(float)
        }
    )

In [36]:
class_obj = data["GET_STATS_DATA"]["STATISTICAL_DATA"]["CLASS_INF"]["CLASS_OBJ"]
for co in class_obj:
    class_entries = co["CLASS"]
    if isinstance(class_entries, list):
        cls_df = pd.DataFrame(class_entries)
    elif isinstance(class_entries, dict):
        cls_df = pd.DataFrame(pd.Series(class_entries)).T
    else:
        print("想定外のCLASSの型: ", type(class_entries), "\nCLASSの値: ", class_entries)
        continue
    cls_df = (
        cls_df
        .set_index("@code")
        .rename(columns=lambda col: f"{co['@name']}{col.lstrip('@')}")
    )
    value_df = (
        value_df
        .merge(cls_df, left_on=co["@id"], right_index=True, how="left")
        .rename(columns={co["@id"]: f"{co['@name']}code"})
    )
value_df.columns

Index(['表章項目code', '用途分類code', '世帯区分code', '世帯主の年齢階級code', '地域区分code',
       '時間軸（月次）code', 'unit', 'value', '表章項目name', '表章項目level', '用途分類name',
       '用途分類level', '用途分類unit', '用途分類parentCode', '世帯区分name', '世帯区分level',
       '世帯主の年齢階級name', '世帯主の年齢階級level', '地域区分name', '地域区分level', '時間軸（月次）name',
       '時間軸（月次）level'],
      dtype='str')

In [37]:
attr_map = {"value": "値", "code": "コード", "name": "", "level": "階層レベル", 
    "unit": "単位", "parentCode": "親コード", "addInf": "追加情報", "tab": "表章項目", 
    "cat": "分類", "area": "地域", "time": "時間軸", "annotation": "注釈記号"  
}
def _convert(c):
    for k, v in attr_map.items():
        if k in c:
            return c.replace(k, v)
    return c

value_df = value_df.rename(columns=_convert)

In [38]:
value_df.head(1).T

,0
表章項目コード,01
用途分類コード,021
世帯区分コード,04
世帯主の年齢階級コード,A00
地域区分コード,00000
時間軸（月次）コード,2020000101
単位,円
値,458814.0
表章項目,金額
表章項目階層レベル,


In [39]:
from estat import get_metainfo, get_statsdata, cleaning_statsdata, colname_to_japanese, create_hierarchy_dataframe

In [40]:
statsDataId = "0002070010"
meta = get_metainfo(appId, statsDataId)
metadata = meta["GET_META_INFO"]["METADATA_INF"]
total_num = metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
total_num

3916136

In [41]:
%%time
data = get_statsdata(appId, statsDataId)
data["GET_STATS_DATA"]["STATISTICAL_DATA"]["RESULT_INF"]["NEXT_KEY"]

CPU times: user 361 ms, sys: 112 ms, total: 473 ms
Wall time: 13.9 s


100001

In [43]:
%%time
dfs = []
dfs.append(colname_to_japanese(cleaning_statsdata(data)))
max_position = 500000
while "NEXT_KEY" in data["GET_STATS_DATA"]["STATISTICAL_DATA"]["RESULT_INF"]:
    start_position = data["GET_STATS_DATA"]["STATISTICAL_DATA"]["RESULT_INF"].get("NEXT_KEY")
    print("NEXT_KEY: ", start_position)
    if start_position > max_position:
        break
    data = get_statsdata(appId, statsDataId, params={"startPosition": start_position})
    dfs.append(colname_to_japanese(cleaning_statsdata(data)))

df = pd.concat(dfs)
df.shape

NEXT_KEY:  500001
CPU times: user 1.1 s, sys: 5.32 ms, total: 1.11 s
Wall time: 1.11 s


(100000, 22)

In [44]:
indicator_url = "https://dashboard.e-stat.go.jp/api/1.0/Json/getIndicatorInfo"
indicator_params = {"StatName": "家計調査"}
indicator_res = requests.get(indicator_url, params=indicator_params)
indicator = indicator_res.json()
indicator_metadata = indicator["GET_META_INDICATOR_INF"]["METADATA_INF"]
indicator_classobj = indicator_metadata["CLASS_INF"]["CLASS_OBJ"][0]
print(indicator_classobj["@name"], indicator_classobj["@code"])

二人以上の世帯　消費支出 0704010101000010000


In [45]:
data_url = "https://dashboard.e-stat.go.jp/api/1.0/Json/getData"
data_params = {"IndicatorCode": "0704010101000010000", "TimeFrom": "20200100"}
data_res = requests.get(data_url, params=data_params)
data = data_res.json()
data["GET_STATS"]["STATISTICAL_DATA"]["DATA_INF"]["DATA_OBJ"][:2]

[{'VALUE': {'@indicator': '0704010101000010000',
   '@unit': '116',
   '@stat': '00200561',
   '@regionCode': '00000',
   '@time': '20200100',
   '@cycle': '1',
   '@regionRank': '2',
   '@isSeasonal': '1',
   '@isProvisional': '0',
   '$': '287173'}},
 {'VALUE': {'@indicator': '0704010101000010000',
   '@unit': '116',
   '@stat': '00200561',
   '@regionCode': '01100',
   '@time': '20200100',
   '@cycle': '1',
   '@regionRank': '4',
   '@isSeasonal': '1',
   '@isProvisional': '0',
   '$': '274960'}}]

In [46]:
data_df = pd.DataFrame(
    [d["VALUE"] for d in data["GET_STATS"]["STATISTICAL_DATA"]["DATA_INF"]["DATA_OBJ"]]
)
data_df = data_df.rename(
    columns=lambda col: col.lstrip("@").replace("$", "value")
)
data_df.head()

,indicator,unit,stat,regionCode,time,cycle,regionRank,isSeasonal,isProvisional,value
0,0704010101000010000,116,00200561,00000,20200100,1,2,1,0,287173
1,0704010101000010000,116,00200561,01100,20200100,1,4,1,0,274960
2,0704010101000010000,116,00200561,02201,20200100,1,4,1,0,243762
3,0704010101000010000,116,00200561,03201,20200100,1,4,1,0,270266
4,0704010101000010000,116,00200561,04100,20200100,1,4,1,0,295469


In [47]:
region_url = "https://dashboard.e-stat.go.jp/api/1.0/Json/getRegionInfo"
region_res = requests.get(region_url)
region = region_res.json()
region_dfs = []
region_co = region["GET_META_REGION_INF"]["METADATA_INF"]["CLASS_INF"]["CLASS_OBJ"]
for c in region_co:
    region_dfs += [pd.DataFrame(c["CLASS"])]
region_df = pd.concat(region_dfs)
region_df = region_df.rename(
    columns=lambda col: col.lstrip("@").replace("name", "地域")
)
region_df.tail()

,regionCode,地域,level,hiragana,fromDate,toDate
3,47374,(旧)伊良部村,13,いらぶそん,197206,198204
4,47374,(旧)伊良部町,12,いらぶちょう,198204,200510
5,47375,多良間村,13,たらまそん,197206,999912
0,47381,竹富町,12,たけとみちょう,197205,999912
1,47382,与那国町,12,よなぐにちょう,197205,999912


In [48]:
data_df = data_df.merge(region_df[["regionCode", "地域"]], on="regionCode", how="left")

In [49]:
data_df = data_df.assign(**{
    "年":  lambda df: df["time"].astype(int) // 10000,
    "月":  lambda df: df["time"].str[4:6].astype(int),
    "年月": lambda df: pd.to_datetime(
        df["年"].astype(str) + "-" +
        df["月"].astype(str) + "-01"
    )
})
data_df.head()

,indicator,unit,stat,regionCode,time,cycle,regionRank,isSeasonal,isProvisional,value,地域,年,月,年月
0,0704010101000010000,116,00200561,00000,20200100,1,2,1,0,287173,全国,2020,1,2020-01-01
1,0704010101000010000,116,00200561,01100,20200100,1,4,1,0,274960,札幌市,2020,1,2020-01-01
2,0704010101000010000,116,00200561,02201,20200100,1,4,1,0,243762,青森市,2020,1,2020-01-01
3,0704010101000010000,116,00200561,03201,20200100,1,4,1,0,270266,盛岡市,2020,1,2020-01-01
4,0704010101000010000,116,00200561,04100,20200100,1,4,1,0,295469,仙台市,2020,1,2020-01-01


In [50]:
socialevent_url = "https://dashboard.e-stat.go.jp/api/1.0/Json/getSocialEventInfo"
socialevent_params = {"TimeFrom": "20200100"}
socialevent_res = requests.get(socialevent_url, params=socialevent_params)
socialevent = socialevent_res.json()
socialevent_df = pd.json_normalize(socialevent, 
    record_path=["GET_META_SOCIAL_INFO", "METADATA_INF", "CLASS_INF", "CLASS_OBJ"]
)
socialevent_df.drop("CLASS", axis=1)

,@code,@name,@level,@fromTime,@toTime
0,00089,令和２年７月豪雨等,2,2020年7月,2020年7月
1,00090,新型コロナウイルス感染症緊急事態宣言,3,2020年4月,2020年5月
2,00091,新型コロナウイルス感染症緊急事態宣言,3,2021年1月,2021年3月
3,00092,新型コロナウイルス感染症緊急事態宣言,3,2021年4月,2021年6月
4,00093,新型コロナウイルス感染症緊急事態宣言,3,2021年7月,2021年9月
5,00094,ロシアのウクライナ侵略始まる,3,2022年2月,2022年2月
